In [ ]:
pip install -U transformers datasets accelerate peft trl bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 17.5 MB/s eta 0:00:00
  Using cach

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import gc
import os

import torch
from datasets import load_dataset
from google.colab import userdata
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    AutoModelForSeq2SeqLM,
    GenerationConfig,
)

In [ ]:
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 38.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 7.3 MB/s eta 0:00:00


In [ ]:
pip install tiktoken

In [ ]:
model = "glp500/OrpoLlama-3-8B-Instruct2"
#max_seq_length = 8000

# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=False,
    bnb_8bit_quant_type="nf4",
    bnb_8bit_compute_dtype=torch_dtype,
    bnb_8bit_use_double_quant=False,
)

# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model, padding_side="left")

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model,
    #quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation,
    torch_dtype=torch_dtype,
)





Unused kwargs: ['bnb_8bit_quant_type', 'bnb_8bit_compute_dtype', 'bnb_8bit_use_double_quant']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/419 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

In [ ]:
generation_config = GenerationConfig(

     #num_beams=3, num_beam_groups=3, diversity_penalty=0.1
     do_sample=True, num_beams=5,penalty_alpha=0.6
     #do_sample=False, num_beams=5, penalty_alpha=0.6


)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
def strip_before_response(text):
    # Check if the input is a list
    if isinstance(text, list):
        # Apply the function to each element in the list
        return [strip_before_response(item) for item in text]

    # Find the index of the substring "Response:\n"
    index = text.find("Response:\n")

    # If the substring is found, extract the text after it
    if index != -1:
        return text[index + len("Response:\n"):]
    else:
        return text  # If the substring is not found, return the original text




In [ ]:
def token_count(text):
    tokenized_input = tokenizer(text, return_tensors="pt")
    token_count = tokenized_input.input_ids.size(1)
    return token_count

def get_max_new_tokens(text):
    return int(token_count(text) - token_count(text)*0.25)

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer

# Load the CSV file
csv_file_path = "/content/drive/Othercomputers/My MacBook Pro/Thesis_Project/Data/training:test set/test.csv"
df = pd.read_csv(csv_file_path)

# Prepare the inferences
inferences = df["input"].tolist()

# Run the inferences
results = []
for input_text in inferences:

    num_tokens = get_max_new_tokens(input_text)

    inputs = tokenizer(
    [
    alpaca_prompt.format(
        "You are a helpful assistant designed to help users translate text from early modern dutch to english. Here are some detailed instructions for translation of sentences from early modern dutch to modern english: (1) Retain the exact sentence structure of the original early modern Dutch text, (2) Keep the sentence length to the same as the original, (3) Preserve all proper nouns (names, dates, events, places, objects) exactly as they appear in the original text, (4) Keep any specific historical or cultural references unchanged, (5) Use contemporary English while retaining the formal or archaic tone of the original early modern Dutch text where appropriate, (6) Avoid contemporary slang or colloquial expressions and (7) Please only include the english translation of the early modern dutch text in your response. Please translate the following early modern Dutch text sentence into contemporary English text: ", # instruction
        str(input_text), # input
        "", # output - leave this blank for generation!
    )
    ], return_tensors = "pt").to("cuda")

    #outputs = model.generate(**inputs, assistant_model=model)
    outputs = model.generate(**inputs, generation_config=generation_config, max_new_tokens=num_tokens)
    #tokenizer.batch_decode(outputs,skip_special_tokens=True)

    text = strip_before_response(tokenizer.batch_decode(outputs,skip_special_tokens=True))
    results.append(text)

# Optionally, save the results to a new CSV file
df["Llama 3 ORPO"] = results
output_csv_file_path = "/content/drive/Othercomputers/My MacBook Pro/Thesis_Project/Data/translation_test/test_results.csv"
df.to_csv(output_csv_file_path, index=False)

print("Inferences completed and saved to:", output_csv_file_path)

Inferences completed and saved to: /content/drive/Othercomputers/My MacBook Pro/Thesis_Project/Data/translation_test/test_results.csv


The code blocks below is there to run one-shot inferences against using one string of input text. Using this method, it is easier to tweak the generation_config parameters and observe the changes in generated outputs from the model.

In [ ]:
!pip install evaluate
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.6 MB/s eta 0:00:00


In [ ]:
test_text = "Compareerde voor de naergenoemde gecommitteerde leeden uijt den E: agtb: Raed van Justitie deser steede, den heijdens Caniaen Comen geboortig van Coeroernaddoe oud 32: jaeren, onderdaen van den koning van Cochim, den welken door vertaeling van den ondertolk Balthazar den Brouwer onder vertooninge van de gev:s Ittij en Ittinaen, /die hij deposant betuijgt beijde de eijgenste t’wesen over wien hij thans is klagende:/ klagender wijse heeft tekennen gegeven dat hij deposant met boven gem: Ittij hem in swagerschap bestaende van voorm: Coeroernaddoe na Illoewene opsijn gedaen versoek bij sijn broeder Conden is gegaen, dog denselven niet te huijs gevonden hebbende, sijnse met hun beijde voorts na Paloertij vertrocken, alwaer sij ontmoeten hebben den aldaer woonende Canniaen hem deposant bij naeme onbekend benevens den tweeden gev: in desen Ittinaen; dat wijders sij met hun vieren van gem: Paloertij na destad sijn gegaen, daer komende begaven sij gesamentlijk met den deposant na twee â drie huijsen, en eijndel: na de groote schagerije, dat ondertussen den deposant ook gesien heeft voors: Caniaen een ola gesch: en deselve aen des deposants swager Ittij overgegeven heeft, niet wetende ten wat eijnde, dat binnen de schagerije gekomen sijnde, den deposant verstaen heeft hij verkogt zouden werden, als wanneer den deposant daer tegens heeft geprotesteert, met betuijging van geen slaef te wesen maer een vrijgeboorene, dat vervolgens sij tesaemen door seker Toepas na den E: fiscael sijn gebragt, aldaer g’examineerd, en voorts na ‘t gevangen huijs gesonden geworden."

num_tokens = get_max_new_tokens(test_text)

inputs = tokenizer(
[
    alpaca_prompt.format(
        "You are a helpful assistant designed to help users translate text from early modern dutch to english. Here are some detailed instructions for translation of sentences from early modern dutch to modern english: (1) Retain the exact sentence structure of the original early modern Dutch text, (2) Keep the sentence length to the same as the original, (3) Preserve all proper nouns (names, dates, events, places, objects) exactly as they appear in the original text, (4) Keep any specific historical or cultural references unchanged, (5) Use contemporary English while retaining the formal or archaic tone of the original early modern Dutch text where appropriate, (6) Avoid contemporary slang or colloquial expressions and (7) Please only include the english translation of the early modern dutch text in your response. Please translate the following early modern Dutch text sentence into contemporary English text: ", # instruction
        test_text, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, generation_config=generation_config,max_new_tokens=num_tokens)

text = strip_before_response(tokenizer.batch_decode(outputs,skip_special_tokens=True))
print(text)
text = str(text)
from evaluate import load
bertscore = load("bertscore")
predictions = [text]
references = ["Appeared before the undersigned delegated members from the Honorable Court of Justice of this city, the heathen Caniaen Comen, native of Coeroernaddoe, aged 32 years, subject of the king of Cochim, who by translation of the under-interpreter Balthazar den Brouwer, showing the mentioned Ittij and Ittinaen, /whom he, the deponent, asserts are both the same ones about whom he now is complaining:/ in a complaining manner declared that he, the deponent, together with the aforementioned Ittij, who he stands in brotherhood with, hailing from aforementioned Coeroernaddoe to Illoewene at the request of his brother Conden went, but not finding him at home, he with both of them proceeded to Paloertij, where they met the resident Canniaen whom he, the deponent, did not know by name along with the second mentioned Ittinaen in this case; further that they together from the aforementioned Paloertij went to the city, where arriving they all together with the deponent went to two or three houses, and finally to the great market, that meanwhile the deponent also saw the aforementioned Canniaen handed an ola to the deponent’s brother-in-law Ittij, not knowing for what purpose, that on arriving at the market, the deponent understood that he was to be sold, upon which the deponent protested, asserting not to be a slave but a free-born, that subsequently they together were brought by a certain Toepas to the Honorable fiscal, there examined, and then sent to the prison house."]
results = bertscore.compute(predictions=predictions, references=references, lang="en")
print(results)

#

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


['\nYou are a helpful assistant designed to help users translate text from early modern dutch to english. Here are some detailed instructions for translation of sentences from early modern dutch to english: (1) Retain the exact sentence structure of the original early modern dutch text, (2) Keep the sentence length to the same as the original, (3) Preserve the exact sentence structure of the original early modern dutch text, (4) Preserve the exact sentence structure of the original early modern dutch text, (5) Keeping the sentence length to the same as the original early modern dutch text, (6) Preserve the exact sentence structure of the english translation of the original early modern dutch text into contemporary English text, (7) Please only include the english translation of the early modern dutch text into contemporary English text, (8) Please only include the english translation of the early modern dutch text into contemporary English text, (9) Please only include the english tran

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'precision': [0.7305212020874023], 'recall': [0.7606531381607056], 'f1': [0.7452827095985413], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.42.3)'}


In [ ]:
pred = predictions[0].split()
ref = references[0].split()

print(len(pred))
print(len(ref))

268
261


In [ ]:
test_text = "Waarmeede de deposante haare verleende klagt depositie ijndigende, betuigd derselver inhoud de opregte en suivere waarheijd is behelsende."
ref_text = "With which the deponent concluded her given complaint deposition, attesting that its content contains the sincere and pure truth."

num_tokens = get_max_new_tokens(test_text)
print(num_tokens)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "You are a helpful assistant designed to help users translate text from early modern dutch to english. Here are some detailed instructions for translation of sentences from early modern dutch to modern english: (1) Retain the exact sentence structure of the original early modern Dutch text, (2) Keep the sentence length to the same as the original, (3) Preserve all proper nouns (names, dates, events, places, objects) exactly as they appear in the original text, (4) Keep any specific historical or cultural references unchanged, (5) Use contemporary English while retaining the formal or archaic tone of the original early modern Dutch text where appropriate, (6) Avoid contemporary slang or colloquial expressions and (7) Please only include the english translation of the early modern dutch text in your response. Please translate the following early modern Dutch text sentence into contemporary English text: ", # instruction
        test_text, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, generation_config=generation_config, max_new_tokens=num_tokens)
#tokenizer.batch_decode(outputs,skip_special_tokens=True

text = strip_before_response(tokenizer.batch_decode(outputs,skip_special_tokens=True))
print(text)
text = str(text)
from evaluate import load
bertscore = load("bertscore")
predictions = [text]
references = [ref_text]
results = bertscore.compute(predictions=predictions, references=references, lang="en")
print(results)

#

35
['Please translate the given early modern Dutch text sentence into contemporary English text. The translation should retain the exact sentence structure of the original early modern Dutch text, keep the sentence length the']


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'precision': [0.8200435042381287], 'recall': [0.8247349262237549], 'f1': [0.8223825097084045], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.42.3)'}


In [ ]:
pred = predictions[0].split()
ref = references[0].split()

print(len(pred))
print(len(ref))

64
54
